# British Airwaysの顧客レビュー内容を日本語翻訳
- サーバレスコンピュートを使用します
- Kaggle [Airline Review](https://www.kaggle.com/datasets/chaudharyanshul/airline-reviews)のデータセットを使用します

Note:  
Anshul Chaudhary, & Muskan Risinghani. (2023). Airline Reviews [Data set]. Kaggle.

#### 想定のディレクトリ構成
```
/<catalog_name>
├── airline_reviews                           <- スキーマ
│   ├── bz_reviews_en                         <- テーブル：bzonze/レビュー英語
│   ├── bz_reviews_ja                         <- テーブル：bzonze/レビュー日本語
│   ├── sv_reviews_summaries                  <- テーブル：silver/レビュー要約サマリ
│   ├── sv_reviews_vectors                    <- テーブル：silver/レビュー要約サマリベクトル
│   ├── sv_reviews_clusters                   <- テーブル：silver/レビューカテゴリ（クラスタ別）
│   ├── raw_data                              <- ボリューム
│       ├── EN/BA_AirlineReviews.csv             <- KaggleからダウンロードしたCSVを手動アップロード
│       ├── JA/BA_AirlineReviews.csv             <- 日本語翻訳CSV
```

#### 処理概要
1. レビュー内容を日本語翻訳 -> CSV出力
1. レビュー内容をLLMでサマライズ -> サマリーをベクトルに変換
1. ベクトルをNクラスターに分類 -> カテゴリごとにサマライズしてカテゴリ名を作成 -> ベクトルを2次元でプロット

In [0]:
%run ./00_config

## 1. csvロード＆bronzeテーブル作成

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

# スキーマの定義
schema = StructType([
    StructField("No", IntegerType(), True),
    StructField("OverallRating", StringType(), True),
    StructField("ReviewHeader", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Datetime", StringType(), True),
    StructField("VerifiedReview", StringType(), True),
    StructField("ReviewBody", StringType(), True),
    StructField("TypeOfTraveller", StringType(), True),
    StructField("SeatType", StringType(), True),
    StructField("Route", StringType(), True),
    StructField("DateFlown", StringType(), True),
    StructField("SeatComfort", StringType(), True),
    StructField("CabinStaffService", StringType(), True),
    StructField("GroundService", StringType(), True),
    StructField("ValueForMoney", StringType(), True),
    StructField("Recommended", StringType(), True),
    StructField("Aircraft", StringType(), True),
    StructField("FoodandBeverages", StringType(), True),
    StructField("InflightEntertainment", StringType(), True),
    StructField("WifiandConnectivity", StringType(), True)
])

# read files
df = spark.read.format("csv") \
                .option("multiLine", True) \
                .option("quote", '"') \
                .option("escape", '"') \
                .option("header", True) \
                .schema(schema) \
                .load(f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/EN/")

# write table
df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_reviews_en")

print(df.count())
print(df.columns)
display(df.limit(10))

In [0]:
# %sql
# select TypeOfTraveller, count(*) as cnt
# from bz_reviews_en
# group by 1
# order by 2 desc

In [0]:
# %sql
# select SeatType, count(*) as cnt
# from bz_reviews_en
# group by 1
# order by 2 desc

In [0]:
# %sql
# select Route, count(*) as cnt
# from bz_reviews_en
# group by 1
# order by 2 desc

In [0]:
# %sql
# select Aircraft, count(*) as cnt
# from bz_reviews_en
# group by 1
# order by 2 desc

## 2. レビューを日本語翻訳＆CSV出力

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re
from datetime import datetime

# Datetime用UDF
@udf(returnType=StringType())
def convert_yyyymmdd(date_str: str) -> str:
    try:
        if isinstance(date_str, str):
            date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
            return datetime.strptime(date_str, '%d %B %Y').strftime('%Y-%m-%d')
        else:
            return None
    except Exception as e:
        return None

# DateFlown用UDF
@udf(returnType=StringType())
def convert_yyyymm(date_str: str) -> str:
    try:
        if isinstance(date_str, str):
            return datetime.strptime(date_str, '%B %Y').strftime('%Y-%m')
        else:
            return None
    except Exception as e:
        return None

spark.udf.register("convert_yyyymmdd", convert_yyyymmdd)
spark.udf.register("convert_yyyymm", convert_yyyymm)

In [0]:
df = spark.sql(f"""
WITH prompts AS (
  SELECT
    No,
    OverallRating,
    -- ReviewHeader,
    CONCAT(
      "<前提>あなたは、British Airwaysの顧客Feed Back専門のプロの翻訳アシスタントです。",
      "\n<指示>次のレビューヘッダーを自然な日本語に翻訳して下さい。レビューヘッダー:\n",
      ReviewHeader,
      "\n<注意>",
      "\n・「〜です」「〜ます」などの表現は避け、レビューヘッダーらしい言い切り表現を遵守してください。"
      "\n・翻訳結果のみ出力してください。補足は一切不要です。"
      ) AS ReviewHeader_prompt,
    Name,
    convert_yyyymmdd(Datetime) AS Datetime,           -- '16th November 2023' -> '2023-11-16'
    VerifiedReview,
    -- ReviewBody,
    CONCAT(
      "<前提>あなたは、British Airwaysの顧客Feed Back専門のプロの翻訳アシスタントです。",
      "\n<指示>次のレビュー本文を自然で違和感のない日本語に翻訳して下さい。レビュー本文:\n",
      ReviewBody,
      "\n<注意>",
      "\n・翻訳結果のみ出力してください。補足は一切不要です。"
      ) AS ReviewBody_prompt,
    TypeOfTraveller,
    SeatType,
    Route,
    convert_yyyymm(DateFlown) AS DateFlown,        -- 'November 2023' -> '2023-11'
    SeatComfort,
    CabinStaffService,
    GroundService,
    ValueForMoney,
    Recommended,
    Aircraft,
    FoodandBeverages,
    InflightEntertainment,
    WifiandConnectivity
  FROM {MY_CATALOG}.{MY_SCHEMA}.bz_reviews_en
  -- LIMIT 10
),
ja AS (
  SELECT
    No,
    OverallRating,
    ReviewHeader_prompt,
    ai_query('databricks-claude-3-7-sonnet', ReviewHeader_prompt, failOnError => False).result AS ReviewHeader,
    Name,
    Datetime,
    VerifiedReview,
    ReviewBody_prompt,
    ai_query('databricks-claude-3-7-sonnet', ReviewBody_prompt, failOnError => False).result AS ReviewBody,
    TypeOfTraveller,
    SeatType,
    Route,
    DateFlown,
    SeatComfort,
    CabinStaffService,
    GroundService,
    ValueForMoney,
    Recommended,
    Aircraft,
    FoodandBeverages,
    InflightEntertainment,
    WifiandConnectivity
  FROM prompts
)
SELECT
  No,
  OverallRating,
  ReviewHeader_prompt,
  -- ReviewHeader,
  COALESCE(ReviewHeader, '-') AS ReviewHeader,
  Name,
  Datetime,
  VerifiedReview,
  ReviewBody_prompt,
  ReviewBody,
  COALESCE(ReviewBody, '-') AS ReviewBody,
  TypeOfTraveller,
  SeatType,
  Route,
  DateFlown,
  SeatComfort,
  CabinStaffService,
  GroundService,
  ValueForMoney,
  Recommended,
  Aircraft,
  FoodandBeverages,
  InflightEntertainment,
  WifiandConnectivity
FROM ja
""")

print(df.count())
print(df.columns)
# display(df)
display(df.limit(100))

In [0]:
selected_df = df.select(
    "No",
    "OverallRating",
    "ReviewHeader",
    "Name",
    "Datetime",
    "VerifiedReview",
    "ReviewBody",
    "TypeOfTraveller",
    "SeatType",
    "Route",
    "DateFlown",
    "SeatComfort",
    "CabinStaffService",
    "GroundService",
    "ValueForMoney",
    "Recommended",
    "Aircraft",
    "FoodandBeverages",
    "InflightEntertainment",
    "WifiandConnectivity"
)

# # write table
# selected_df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_reviews_ja")

# export csv files
selected_df.coalesce(1).toPandas().to_csv(f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/JA/BA_AirlineReviews_new.csv", index=False, quoting=1)

print(selected_df.count())
print(selected_df.columns)
display(selected_df.limit(100))

In [0]:
# %sql
# spark.sql(f"""
# SELECT
#   MAX(OverallRating) as max_rate,
#   MIN(OverallRating) as min_rate
# FROM {MY_CATALOG}.{MY_SCHEMA}.bz_reviews_ja
# """).display()